In [3]:
import pandas as pd
import sqlite3

## Instancia de BBDD

In [4]:
# Conectamos con la base de datos chinook.db
connection = sqlite3.connect("data/Bar_TB.db")

# Obtenemos un cursor que utilizaremos para hacer las queries
crsr = connection.cursor()

### Funcion para interactuar con la base de datos

In [5]:
# Con esta función leemos los datos y lo pasamos a un DataFrame de Pandas
def sql_query(query):

    # Ejecuta la query
    crsr.execute(query)

    # Almacena los datos de la query 
    ans = crsr.fetchall()

    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in crsr.description]

    return pd.DataFrame(ans,columns=names)

## Crear las tablas

In [6]:
res = crsr.execute("SELECT name FROM sqlite_master WHERE type='table'")
for name in res:
    print(name[0])

Cerveza


### Tabla Cerveza

In [7]:
query_cerveza = """
CREATE TABLE Cerveza (
    CodC VARCHAR(3),
    Enavase VARCHAR(50),
    Capacidad FLOAT,
    Stock INT
)
"""

crsr.execute(query_cerveza)


OperationalError: table Cerveza already exists

### Tabla Bares

In [9]:
query_bares = """
CREATE TABLE Bares (
    CodB VARCHAR(3),
    Nombre VARCHAR(50),
    Cif VARCHAR(10),
    Localidad VARCHAR(50)
)
"""

crsr.execute(query_bares)

### Tabla Empleados

In [ ]:
query_empleados = """
CREATE TABLE Empleados (
    CodE VARCHAR(2),
    Nombre VARCHAR(50),
    Sueldo(10),
)
"""

crsr.execute(query_empleados)

### Tabla Reparto

## Insertar Datos

In [10]:
# Insertamos datos en la tabla Cerveza
query_insert = """
INSERT INTO Cerveza VALUES
('01', 'Botella', 0.2, 3600),
('02', 'Botella', 0.33, 1200),
('03', 'Lata', 0.33, 2400),
('04', 'Botella', 1, 288),
('05', 'Barril', 60, 30)
"""

crsr.execute(query_insert)

In [11]:
pd.read_sql("SELECT * FROM Cerveza", connection)

,CodC,Enavase,Capacidad,Stock
0,01,Botella,0.20,3600
1,02,Botella,0.33,1200
2,03,Lata,0.33,2400
3,04,Botella,1.00,288
4,05,Barril,60.00,30


In [ ]:
# Insertamos datos en la tabla Reparto
query_insert2 = """
INSERT INTO Reparto (CodE, CodC, Fecha, Cantidad) VALUES
('1', '001', '01', '10/21/05', 240),
(2, 001, 02, 10/21/05, 48),
(3, 003, 01, 10/21/05, 240),
(4, 004, 01, 10/21/05, 240),
(5, 005, 01, 10/21/05, 240)
"""

crsr.execute(query_insert2)


In [ ]:
pd.read_sql("SELECT * FROM Reparto", connection)

## Respuestas